In [7]:
import pandas as pd
import os
import sqlite3
from sqlite3 import Error

In [8]:
from constants import * 
from utils import *

In [9]:
print(DB_PATH)
print(DB_FILE_NAME)

/home/airflow/dags/Lead_scoring_data_pipeline/
lead_scoring_data_cleaning.db


In [10]:
cnx = sqlite3.connect(DB_PATH + DB_FILE_NAME)
df = pd.read_sql(f"SELECT name FROM sqlite_master WHERE type='table'", cnx)
print(df.shape)
df.head()

(7, 1)


,name
0,target
1,predicted_output
2,features
3,loaded_data
4,city_tier_mapped


In [11]:
df = pd.read_sql('select * from interactions_mapped', cnx)
print(df.shape)
df.head()

(238124, 14)


,index,created_date,city_tier,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,referred_lead,app_complete_flag,assistance_interaction,career_interaction,payment_interaction,social_interaction,syllabus_interaction
0,0,2021-07-01 00:08:15,1.0,Level0,others,Level2,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0
1,1,2021-07-01 00:16:43,2.0,Level3,Level0,others,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0
2,2,2021-07-01 00:22:20,1.0,Level3,Level0,Level0,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0
3,3,2021-07-01 00:23:13,1.0,Level1,Level3,others,2.0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,4,2021-07-01 00:28:38,1.0,Level3,Level0,Level0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [12]:
encoded_df = df[FEATURES_TO_ENCODE]
encoded_df.head()

,first_platform_c,first_utm_medium_c,first_utm_source_c
0,Level0,others,Level2
1,Level3,Level0,others
2,Level3,Level0,Level0
3,Level1,Level3,others
4,Level3,Level0,Level0


In [13]:
columns = df.columns.to_list()
print(columns)
print(len(columns))

['index', 'created_date', 'city_tier', 'first_platform_c', 'first_utm_medium_c', 'first_utm_source_c', 'total_leads_droppped', 'referred_lead', 'app_complete_flag', 'assistance_interaction', 'career_interaction', 'payment_interaction', 'social_interaction', 'syllabus_interaction']
14


In [14]:
df_new_encoded = pd.get_dummies(df, columns = FEATURES_TO_ENCODE,drop_first=True)

In [15]:
columns = df_new_encoded.columns.to_list()
print(columns)
print(len(columns))

['index', 'created_date', 'city_tier', 'total_leads_droppped', 'referred_lead', 'app_complete_flag', 'assistance_interaction', 'career_interaction', 'payment_interaction', 'social_interaction', 'syllabus_interaction', 'first_platform_c_Level1', 'first_platform_c_Level2', 'first_platform_c_Level3', 'first_platform_c_Level7', 'first_platform_c_Level8', 'first_platform_c_others', 'first_utm_medium_c_Level1', 'first_utm_medium_c_Level2', 'first_utm_medium_c_Level3', 'first_utm_medium_c_Level7', 'first_utm_medium_c_Level8', 'first_utm_medium_c_others', 'first_utm_source_c_Level1', 'first_utm_source_c_Level2', 'first_utm_source_c_Level3', 'first_utm_source_c_Level7', 'first_utm_source_c_Level8', 'first_utm_source_c_others']
29


In [16]:
db_path=DB_PATH
db_file_name=DB_FILE_NAME
one_hot_encoded_features=ONE_HOT_ENCODED_FEATURES
features_to_encode=FEATURES_TO_ENCODE
model_config=MODEL_CONFIG
experiment=EXPERIMENT
tracking_uri=TRACKING_URI

In [17]:
FEATURES_TO_ENCODE

['first_platform_c', 'first_utm_medium_c', 'first_utm_source_c']

In [18]:
encode_features(db_path, db_file_name, one_hot_encoded_features, features_to_encode)

Data has been extracted successfully from lead_scoring_model_experimentation.
input_data has been saved successfully to table features
input_data has been saved successfully to table target
input data is loaded into features and target tables successfully.


In [19]:
df = pd.read_sql(f"SELECT name FROM sqlite_master WHERE type='table'", cnx)
print(df.shape)
df.head()

(7, 1)


,name
0,predicted_output
1,loaded_data
2,city_tier_mapped
3,categorical_variables_mapped
4,interactions_mapped


In [20]:
cnx = sqlite3.connect(db_path + db_file_name)
X = pd.read_sql('select * from features', cnx)
X.head()

,index,first_platform_c,first_utm_medium_c,first_utm_source_c,total_leads_droppped,city_tier,first_platform_c_Level8,first_platform_c_others,first_platform_c_Level2,first_utm_medium_c_others,first_utm_medium_c_Level0,first_platform_c_Level7,first_platform_c_Level0
0,0,0,0,0,1.0,1.0,0,0,0,1,0,0,1
1,1,0,0,0,1.0,2.0,0,0,0,0,1,0,0
2,2,0,0,0,1.0,1.0,0,0,0,0,1,0,0
3,3,0,0,0,2.0,1.0,0,0,0,0,0,0,0
4,4,0,0,0,1.0,1.0,0,0,0,0,1,0,0


In [21]:
y = pd.read_sql('select * from target', cnx)
y.head()

,index,app_complete_flag
0,0,1
1,1,1
2,2,1
3,3,0
4,4,0


In [22]:
## create sql lite connection and start database

In [23]:
# create a connection and setup a SQLite database with the name "lead_scoring_model_experimentation.db" in 
# 'Assignment/02_training_pipeline/notebook/' location
def create_sqlit_connection(db_path, db_file):
    """ create a database connection to a SQLite database """
    conn = None
    # opening the conncetion for creating the sqlite db
    try:
        conn = sqlite3.connect(db_path + db_file)
        print(sqlite3.version)
    # return an error if connection not established
    except Error as e:
        print(e)
    # closing the connection once the database is created
    finally:
        if conn:
            conn.close()


In [24]:
create_sqlit_connection("/home/mlruns/",r"Lead_scoring_mlflow_production.db")

2.6.0


In [25]:
#mlflow server --backend-store-uri='sqlite:////home/Assignment/02_training_pipeline/scripts/database/Lead_scoring_mlflow_production.db' --default-artifact-root="/home/Assignment/02_training_pipeline/scripts/mlruns/" --port=6006 --host=0.0.0.0

In [26]:
mlflow.set_tracking_uri("http://0.0.0.0:6006")

In [27]:
get_trained_model(db_path, db_file_name, model_config, experiment,tracking_uri)

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
Registered model 'LightGBM' already exists. Creating a new version of this model...
2022/10/24 11:53:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: LightGBM, version 8
Created version '8' of model 'LightGBM'.


get_trained_model has been executed successfully.
